In [1]:
# Importing necessary libraries
import pandas as pd
import numpy as np
import plotly.graph_objs as go
import plotly.express as px
from plotly.offline import plot
from plotly.subplots import make_subplots
import requests
import folium

print('libraries imported!')

libraries imported!


In [2]:
# Importing product_a.csv dataset into pandas DataFrame with first column as index

df_product_a = pd.read_csv('product_a.csv', index_col=0)
df_product_a.head()

,date_w,price,total_vol,plu1,plu2,plu3,bags_t,bags_s,bags_l,bags_lx,type,year,location
0,2016-12-24,1.33,64236.62,1036.74,54454.85,48.16,8696.87,8603.62,93.25,0.0,A,2015,Albany
1,2016-12-17,1.35,54876.98,674.28,44638.81,58.33,9505.56,9408.07,97.49,0.0,A,2015,Albany
2,2016-12-10,0.93,118220.22,794.70,109149.67,130.50,8145.35,8042.21,103.14,0.0,A,2015,Albany
3,2016-12-03,1.08,78992.15,1132.00,71976.41,72.58,5811.16,5677.40,133.76,0.0,A,2015,Albany
4,2016-11-26,1.28,51039.60,941.48,43838.39,75.78,6183.95,5986.26,197.69,0.0,A,2015,Albany


In [3]:
# Converting date_w field to a suitable datetime data type

# Alternative code - df_product_a.date_w.astype('datetime64[ns]')
df_product_a['date_w'] = pd.to_datetime(df_product_a.date_w, format='%Y-%m-%d')
df_product_a.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 18249 entries, 0 to 11
Data columns (total 13 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   date_w     18249 non-null  datetime64[ns]
 1   price      18249 non-null  float64       
 2   total_vol  18249 non-null  float64       
 3   plu1       18249 non-null  float64       
 4   plu2       18249 non-null  float64       
 5   plu3       18249 non-null  float64       
 6   bags_t     18249 non-null  float64       
 7   bags_s     18249 non-null  float64       
 8   bags_l     18249 non-null  float64       
 9   bags_lx    18249 non-null  float64       
 10  type       18249 non-null  object        
 11  year       18249 non-null  int64         
 12  location   18249 non-null  object        
dtypes: datetime64[ns](1), float64(9), int64(1), object(2)
memory usage: 1.9+ MB


In [4]:
# Correcting 'year' field values

df_product_a['year'] = df_product_a.date_w.dt.year
df_product_a.head()

,date_w,price,total_vol,plu1,plu2,plu3,bags_t,bags_s,bags_l,bags_lx,type,year,location
0,2016-12-24,1.33,64236.62,1036.74,54454.85,48.16,8696.87,8603.62,93.25,0.0,A,2016,Albany
1,2016-12-17,1.35,54876.98,674.28,44638.81,58.33,9505.56,9408.07,97.49,0.0,A,2016,Albany
2,2016-12-10,0.93,118220.22,794.70,109149.67,130.50,8145.35,8042.21,103.14,0.0,A,2016,Albany
3,2016-12-03,1.08,78992.15,1132.00,71976.41,72.58,5811.16,5677.40,133.76,0.0,A,2016,Albany
4,2016-11-26,1.28,51039.60,941.48,43838.39,75.78,6183.95,5986.26,197.69,0.0,A,2016,Albany


In [5]:
# Descriptive statistical analysis of 'df_product_a'

# numeric columns to use
columns_to_include = ['price', 'total_vol', 'plu1', 'plu2', 'plu3', 'bags_t', 'bags_s', 'bags_l', 'bags_lx']

df_stats = df_product_a.describe(percentiles=[0.1, 0.2, 0.25, 0.3, 0.4, 0.5, 0.6, 0.7, 0.75, 0.8, 0.9],
                                exclude=[np.object, np.int64, np.datetime64]).transpose()
# add columns - variance, median, mode
# NOTE: For mode, there is a tie between multiple values in 'total_vol' column, I am choosing one arbitrarily
df_stats['var'], df_stats['median'], df_stats['mode'] = df_product_a[columns_to_include].var(), df_product_a[columns_to_include].median(), df_product_a[columns_to_include].mode().iloc[0]
# add 'field_name' column to 'df_stats'
df_stats.index.name = 'field_name'
df_stats.reset_index(inplace=True)

df_stats

,field_name,count,mean,std,min,10%,20%,25%,30%,40%,50%,60%,70%,75%,80%,90%,max,var,median,mode
0,price,18249.0,1.405978,4.026766e-01,0.44,0.930,1.050,1.10,1.150,1.260,1.37,1.480,1.600,1.66,1.740,1.930,3.25,1.621484e-01,1.37,1.15
1,total_vol,18249.0,850644.013009,3.453545e+06,84.56,3896.768,8168.866,10838.58,15181.304,42137.088,107376.76,192430.124,319613.142,432962.29,604868.968,1387045.760,62505646.52,1.192698e+13,107376.76,2038.99
2,plu1,18249.0,293008.424531,1.264989e+06,0.00,94.276,483.362,854.07,1368.118,3261.580,8645.30,31698.078,77991.410,111020.20,152679.068,538385.184,22743616.17,1.600197e+12,8645.30,0.00
3,plu2,18249.0,295154.568356,1.204120e+06,0.00,367.484,1918.532,3008.78,4265.508,10961.994,29061.02,53202.052,98296.168,150206.86,222163.620,500784.552,20470572.61,1.449906e+12,29061.02,0.00
4,plu3,18249.0,22839.735993,1.074641e+05,0.00,0.000,0.000,0.00,0.000,42.704,184.99,768.782,3466.120,6243.42,10972.600,31492.442,2546439.11,1.154853e+10,184.99,0.00
5,bags_t,18249.0,239639.202060,9.862424e+05,0.00,1299.208,3347.674,5088.64,7316.634,16643.260,39743.83,62361.468,88901.748,110783.37,149306.360,442141.928,19373134.37,9.726741e+11,39743.83,0.00
6,bags_s,18249.0,182194.686696,7.461785e+05,0.00,583.110,1686.086,2849.42,4761.328,11502.076,26362.82,46725.754,68884.094,83337.67,104537.112,354266.852,13384586.80,5.567824e+11,26362.82,0.00
7,bags_l,18249.0,54338.088145,2.439660e+05,0.00,0.000,30.918,127.47,329.944,1105.110,2647.71,6087.796,14392.206,22029.25,34350.986,94295.338,5719096.61,5.951939e+10,2647.71,0.00
8,bags_lx,18249.0,3106.426507,1.769289e+04,0.00,0.000,0.000,0.00,0.000,0.000,0.00,0.000,20.252,132.50,557.114,3688.912,551693.65,3.130385e+08,0.00,0.00


In [6]:
# Creating a Pearson correlation matrix

df_coef_p = df_product_a.corr()
df_coef_p.style.background_gradient(cmap='coolwarm').set_precision(2)

,price,total_vol,plu1,plu2,plu3,bags_t,bags_s,bags_l,bags_lx,year
price,1.00,-0.19,-0.21,-0.17,-0.18,-0.18,-0.17,-0.17,-0.12,0.10
total_vol,-0.19,1.00,0.98,0.97,0.87,0.96,0.97,0.88,0.75,0.02
plu1,-0.21,0.98,1.00,0.93,0.83,0.92,0.93,0.84,0.70,0.00
plu2,-0.17,0.97,0.93,1.00,0.89,0.91,0.92,0.81,0.69,-0.01
plu3,-0.18,0.87,0.83,0.89,1.00,0.79,0.80,0.70,0.68,-0.04
bags_t,-0.18,0.96,0.92,0.91,0.79,1.00,0.99,0.94,0.80,0.07
bags_s,-0.17,0.97,0.93,0.92,0.80,0.99,1.00,0.90,0.81,0.06
bags_l,-0.17,0.88,0.84,0.81,0.70,0.94,0.90,1.00,0.71,0.09
bags_lx,-0.12,0.75,0.70,0.69,0.68,0.80,0.81,0.71,1.00,0.08
year,0.10,0.02,0.00,-0.01,-0.04,0.07,0.06,0.09,0.08,1.00


In [7]:
# Creating a Spearman Rank correlation matrix

df_coef_sp = df_product_a.corr(method='spearman')
df_coef_sp.style.background_gradient(cmap='coolwarm').set_precision(2)

,price,total_vol,plu1,plu2,plu3,bags_t,bags_s,bags_l,bags_lx,year
price,1.00,-0.61,-0.60,-0.51,-0.53,-0.60,-0.54,-0.51,-0.43,0.11
total_vol,-0.61,1.00,0.90,0.94,0.83,0.95,0.93,0.70,0.66,0.09
plu1,-0.60,0.90,1.00,0.79,0.75,0.84,0.82,0.64,0.62,0.01
plu2,-0.51,0.94,0.79,1.00,0.82,0.86,0.83,0.64,0.63,0.01
plu3,-0.53,0.83,0.75,0.82,1.00,0.78,0.78,0.57,0.65,-0.04
bags_t,-0.60,0.95,0.84,0.86,0.78,1.00,0.96,0.75,0.65,0.21
bags_s,-0.54,0.93,0.82,0.83,0.78,0.96,1.00,0.60,0.64,0.18
bags_l,-0.51,0.70,0.64,0.64,0.57,0.75,0.60,1.00,0.50,0.19
bags_lx,-0.43,0.66,0.62,0.63,0.65,0.65,0.64,0.50,1.00,0.17
year,0.11,0.09,0.01,0.01,-0.04,0.21,0.18,0.19,0.17,1.00


In [8]:
# Create a Plotly scatter matrix plot for df_product_a

# What is 'trace' in plotly? 
# A trace is just the name we give a collection of data and the specifications of which we want that data plotted

fig = px.scatter_matrix(df_product_a.drop(['date_w'], axis=1), 
                        width=1500,
                        height=1500,
                        opacity=0.4, 
                        title='Scatter matrix plot for df_product_a'
                       )
fig.update_layout(title_x=0.5)
# open the figure in another tab as it is large in size
plot(fig, filename='scatter_matrix_plot.html')

'scatter_matrix_plot.html'

In [9]:
# Creating weekly and monthly time-series graphs of the numeric fields

# convert the dataframe into a time-series format
df_product_a_ts = df_product_a.drop(['type', 'year', 'location'], axis=1).set_index('date_w').sort_index()

# downsample the data to weekly data points using average values for each week (weekly)
df_product_a_weekly = df_product_a_ts.resample('W').mean().dropna()
# downsample the data to monthly data points using average values for each month (monthly)
df_product_a_monthly = df_product_a_ts.resample('M').mean().dropna()

# creating traces
fig = make_subplots(rows=2, cols=1)

for col in df_product_a_ts.columns:
    # weekly
    fig.add_trace(go.Scatter(x=df_product_a_weekly.index, y=df_product_a_weekly[col],
                        mode='lines+markers',
                        name='{} weekly'.format(col)), row=1, col=1)
    
    # monthly
    fig.add_trace(go.Scatter(x=df_product_a_monthly.index, y=df_product_a_monthly[col],
                        mode='lines+markers',
                        name='{} monthly'.format(col)), row=2, col=1)

fig.update_layout(height=1400, title_text='Weekly and monthly time-series graphs', title_x=0.5)

plot(fig, filename='weekly_monthly_timeseries_graphs.html')

'weekly_monthly_timeseries_graphs.html'

In [10]:
# Creating year based location and type bar charts for the total volume

# years to analyse
years = [2016, 2017, 2018, 2019]

# 4 subplots - one for each year
fig = make_subplots(rows=4, cols=1, vertical_spacing=0.12)

for idx, year in enumerate(years):
    # type 'A'
    df_product_a_A = df_product_a.loc[((df_product_a.year == year) & (df_product_a.type == 'A')), ['total_vol', 'location', 'type']]
    # grouped by locations
    _A = df_product_a_A.groupby('location')['total_vol'].sum()
    # type 'C'
    df_product_a_C = df_product_a.loc[((df_product_a.year == year) & (df_product_a.type == 'C')), ['total_vol', 'location', 'type']]
    # grouped by locations
    _C = df_product_a_C.groupby('location')['total_vol'].sum()
    
    fig.add_trace(go.Bar(name='Type A', x=_A.index, y=_A), idx + 1, 1)
    fig.add_trace(go.Bar(name='Type C', x=_C.index, y=_C), idx + 1, 1)

    # plotting stacked bar chart
    fig.update_layout(barmode='stack', height=1500)

plot(fig, filename='total_vol_by_location_type.html')

'total_vol_by_location_type.html'

In [11]:
# Geolocations API service credentials

GOOGLE_API_KEY = 'Your API key'

In [12]:
# Functions to retrieve geo coordinates (i.e. longitudes and latitudes) - Google, Here and ArcGic API services

def get_geo_coordinates_from_google(address:str, connection_params:dict):
    base_url = 'https://maps.googleapis.com/maps/api/geocode'
    url = '{}/{}?address={}&key={}'.format(base_url,
                          connection_params['output_format'],
                          address,
                          connection_params['api_key']    
                         )
    
    # make the GET request
    results = requests.get(url).json()
    
    # check if codes were successfully obtained or not
    if results['status'] == 'ZERO_RESULTS':
        return None
    
    location = results['results'][0]['geometry']['location']
    return {
        'longitude' : location['lng'],
        'latitude' : location['lat'],
    }

# testing functions
# Google API parameters
connection_params = {
    'output_format': 'json',
    'api_key': GOOGLE_API_KEY
}
address = 'TotalUS,+US'

lng_lat = get_geo_coordinates_from_google(address, connection_params)

In [13]:
# getting geocodes for all the unique location values in 'product_a' dataset

connection_params = {
    'output_format': 'json',
    'api_key': GOOGLE_API_KEY
}

location_geocodes = {addr: get_geo_coordinates_from_google('{},+US'.format(addr), connection_params) for addr in df_product_a['location'].unique()}

In [14]:
# Visualising data with Folium map

# USA coordinates
latitude = 37.2754919
longitude = -104.6582933
# create map of United States using latitude and longitude values
map_us = folium.Map(location=[latitude, longitude], zoom_start=4)

# add markers to map
for address in location_geocodes.keys():
    # check if location geocodes are present
    if location_geocodes[address] != None:
        lat = location_geocodes[address]['latitude']
        lng = location_geocodes[address]['longitude']

        address = address + ', US'
        label = folium.Popup(address, parse_html=True)
        # add to the map
        folium.CircleMarker(
            [lat, lng],
            radius=5,
            popup=label,
            color='blue',
            fill=True,
            fill_color='#3186cc',
            fill_opacity=0.7,
            parse_html=False).add_to(map_us)
    else:
        print('"{}" geocodes not found :('. format(address+', US'))
    
map_us

"TotalUS, US" geocodes not found :(
"WestTexNewMexico, US" geocodes not found :(
